# MA_DL : Codierung 64bit

In [16]:
!pip install facenet-pytorch # due to issues with the google cloud service

In [17]:
import torch
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.models as models
from PIL import Image

from facenet_pytorch import MTCNN, InceptionResnetV1

dtype = torch.cuda.FloatTensor

## Model

In [18]:
class MultiLabel(nn.Sequential):
    def __init__(self, input_dim=512, output_dim=64):
        super(MultiLabel, self).__init__()
        self.l1 = nn.Linear(input_dim, input_dim)
        self.l2 = nn.Linear(input_dim, input_dim)
        self.l3 = nn.Linear(input_dim, input_dim)
        self.l4 = nn.Linear(input_dim, output_dim)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.dropout(F.relu(self.l1(x)))
        x = self.dropout(F.relu(self.l2(x)))
        x = self.dropout(F.relu(self.l3(x)))
        x = self.l4(x)
        return torch.sigmoid(x)

In [19]:
model = MultiLabel()
model.load_state_dict(torch.load('dl64b_v6_new_300.pt'))
model.cuda()

MultiLabel(
  (l1): Linear(in_features=512, out_features=512, bias=True)
  (l2): Linear(in_features=512, out_features=512, bias=True)
  (l3): Linear(in_features=512, out_features=512, bias=True)
  (l4): Linear(in_features=512, out_features=64, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

## Data

In [20]:
scaler = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()
to_image = transforms.ToPILImage()
transform = transforms.Compose([
    scaler, 
    to_tensor,
])

In [21]:
train_ds = datasets.ImageFolder('../../../data/small_data_new/train/', transform=transform)
train_ds.idx_to_class = {i:c for c, i in train_ds.class_to_idx.items()}
train_dl = DataLoader(train_ds, batch_size=1, num_workers=4)

valid_ds = datasets.ImageFolder('../../../data/small_data_new/valid/', transform=transform)
valid_ds.idx_to_class = {i:c for c, i in valid_ds.class_to_idx.items()}
valid_dl = DataLoader(valid_ds, batch_size=1, num_workers=4)

## Gesichtserkennung

In [22]:
mtcnn = MTCNN(
    image_size=160,
    thresholds=[0.6, 0.7, 0.7],
#        factor=0.709,
#        prewhiten=True,
    keep_all=True,
    device=torch.device('cuda')
    )

## Embeddings 

In [23]:
# Alternativ resnet
face_resnet = InceptionResnetV1(pretrained='casia-webface').eval()

## Training

In [24]:
def calculate_mistakes(pred, target):
    #pred = pred[0]
#    print('calculating mistakes : ')
#    print('   pred: ' + str(pred.shape))
#    print('   target: ' + str(target.shape))
    
    mistakes = 0
    
    if len(pred) != len(target):
        raise Exception('sizes of both tensors must match')
        
    for x,y in zip(pred, target):
        if round(x.item()) != y.item():
            mistakes = mistakes + 1
        
    return mistakes

In [25]:
loss_func = nn.BCELoss() #MultiLabelSoftMarginLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2, weight_decay=1e-5, momentum=0.9) #weight_decay=1e-4 

In [26]:
import matplotlib.pyplot as plt

In [29]:
epochs = 1000
for e in range(epochs):
    print("======================")
    print("Epoch : " + str(e))
    epoch_loss = 0
    epoch_mistakes = 0
    train_size = len(train_dl)
    
    # activate train mode
    model.train()

    for index, (data, target) in enumerate(train_dl):
        faces, prob = mtcnn(to_image(data[0]), return_prob=True)
        target_t = torch.cuda.FloatTensor([int(x) for x in train_ds.idx_to_class[target[0].item()]])

        emb = face_resnet(faces)    
        data_v   = Variable(emb[0], requires_grad=False).type(dtype)
        target_v = Variable(target_t, requires_grad=False).type(dtype)
        
        pred = model.forward(data_v)
        optimizer.zero_grad()
        
        # calculate loss
        loss = loss_func(pred, target_v.float())
        epoch_loss = epoch_loss + loss
        # back prop
        loss.backward()
        optimizer.step()
        
        epoch_mistakes = epoch_mistakes + calculate_mistakes(pred, target_t)
        
    print("Train loss : " + str(epoch_loss.item()))
    print("Average errors : " + str(epoch_mistakes/train_size))
    
    # activate eval mode
    model.eval()
    
    valid_loss = 0
    valid_mistakes = 0
    valid_size = len(valid_dl)
    for index, (data, target) in enumerate(valid_dl):
        faces, prob = mtcnn(to_image(data[0]), return_prob=True)
        target_t = torch.FloatTensor([int(x) for x in valid_ds.idx_to_class[target[0].item()]])
        
        emb = face_resnet(faces) 
        data_v   = Variable(emb[0], requires_grad=False).type(dtype)
        target_v = Variable(target_t, requires_grad=False).type(dtype)

        pred = model.forward(data_v)
        loss = loss_func(pred, target_v.float())
        valid_loss = valid_loss + loss
        valid_mistakes = valid_mistakes + calculate_mistakes(pred, target_t)
        
        
    print("Valid loss : " + str(valid_loss.item()))
    print("Average valid errors : " + str(valid_mistakes/valid_size))

Epoch : 0
Train loss : 29.272371292114258
Average errors : 2.2191358024691357
Valid loss : 6.17915153503418
Average valid errors : 3.0476190476190474
Epoch : 1
Train loss : 26.16469383239746
Average errors : 1.9598765432098766
Valid loss : 5.446869850158691
Average valid errors : 3.0238095238095237
Epoch : 2
Train loss : 26.942073822021484
Average errors : 2.212962962962963
Valid loss : 5.810749053955078
Average valid errors : 2.619047619047619
Epoch : 3
Train loss : 21.6164608001709
Average errors : 1.4753086419753085
Valid loss : 5.911107063293457
Average valid errors : 2.8333333333333335
Epoch : 4
Train loss : 24.995418548583984
Average errors : 1.7962962962962963
Valid loss : 5.958038806915283
Average valid errors : 2.5238095238095237
Epoch : 5
Train loss : 25.563682556152344
Average errors : 2.0123456790123457
Valid loss : 6.495685577392578
Average valid errors : 2.8095238095238093
Epoch : 6
Train loss : 22.928693771362305
Average errors : 1.617283950617284
Valid loss : 6.36897706

Valid loss : 5.843294143676758
Average valid errors : 2.238095238095238
Epoch : 50
Train loss : 22.603445053100586
Average errors : 1.771604938271605
Valid loss : 5.719936370849609
Average valid errors : 1.8571428571428572
Epoch : 51
Train loss : 20.62860107421875
Average errors : 1.4320987654320987
Valid loss : 5.487625598907471
Average valid errors : 2.0714285714285716
Epoch : 52
Train loss : 20.40578269958496
Average errors : 1.5
Valid loss : 4.831586837768555
Average valid errors : 2.4047619047619047
Epoch : 53
Train loss : 19.76136016845703
Average errors : 1.478395061728395
Valid loss : 5.044784069061279
Average valid errors : 2.2142857142857144
Epoch : 54
Train loss : 21.728740692138672
Average errors : 1.6759259259259258
Valid loss : 4.872158527374268
Average valid errors : 1.7619047619047619
Epoch : 55
Train loss : 19.040210723876953
Average errors : 1.3518518518518519
Valid loss : 5.234772682189941
Average valid errors : 2.5238095238095237
Epoch : 56
Train loss : 24.744436264

KeyboardInterrupt: 

In [30]:
torch.save(model.state_dict(), 'dl64b_v6_new_606.pt')

## Testing 

In [62]:
def tensor_to_binary(tensor):
    if tensor.dim() != 2:
        raise ValueException('tensor must have two dimensions')

    for i, x in enumerate(tensor):
        for j, y in enumerate(x) :
            if y.item() < 0.5:
                tensor[i][j] = 0
            elif y.item() >= 0.5:
                tensor[i][j] = 1

    return tensor

In [65]:
for i, t in train_ds:
    print(i.shape)
    faces, prob = mtcnn(to_image(i), return_prob=True)
    target_t = torch.FloatTensor([int(x) for x in valid_ds.idx_to_class[target[0].item()]])
        
    emb = face_resnet(faces) 
    data_v   = Variable(emb[0], requires_grad=False).type(dtype)
    target_v = Variable(target_t, requires_grad=False).type(dtype)

    pred = model.forward(data_v)
    print(pred.shape)
    print(tensor_to_binary(pred.unsqueeze(0)))

torch.Size([3, 224, 224])
torch.Size([64])
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1.,
         1., 0., 1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1., 1.,
         0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 1., 0.,
         0., 0., 0., 1., 1., 1., 0., 0., 0., 1.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1.,
         1., 0., 1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1., 1.,
         0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 1., 0.,
         0., 0., 0., 1., 1., 1., 0., 0., 0., 1.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1.,
         1., 0., 1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1., 1.,
         0., 1.

torch.Size([64])
tensor([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0.,
         0., 1., 1., 0., 1., 1., 1., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1., 0.,
         0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 1., 0., 0., 1., 1.,
         1., 1., 1., 1., 1., 0., 1., 0., 0., 0.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0.,
         0., 1., 1., 0., 1., 1., 1., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1., 0.,
         0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 1., 0., 0., 1., 1.,
         1., 1., 1., 1., 1., 0., 1., 0., 0., 0.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0.,
         0., 1., 1., 0., 1., 1., 1., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1., 0.,
         0., 1., 0., 0., 0., 0., 1., 0., 

torch.Size([64])
tensor([[0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1.,
         1., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 1., 0., 1.,
         1., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0.,
         0., 1., 0., 0., 1., 0., 1., 1., 0., 1.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[0., 0., 0., 1., 1., 0., 0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0.,
         0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1.,
         0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0., 0., 0.,
         0., 0., 0., 1., 0., 1., 1., 1., 0., 0.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[0., 0., 0., 1., 1., 0., 0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0.,
         0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1.,
         0., 0., 1., 1., 1., 1., 1., 1., 

torch.Size([64])
tensor([[0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0., 1.,
         0., 1., 1., 1., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0.,
         0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 0., 0.,
         0., 1., 0., 1., 1., 0., 0., 1., 0., 0.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0.,
         0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 1., 1., 1., 1.,
         1., 1., 1., 1., 0., 1., 0., 0., 0., 1.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0.,
         0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 1., 

torch.Size([64])
tensor([[0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 1., 1.,
         0., 0., 0., 0., 1., 0., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0.,
         1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 1.,
         1., 0., 0., 1., 0., 1., 0., 1., 0., 0.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0., 0., 1., 1.,
         1., 1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0.,
         0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1.,
         1., 1., 0., 1., 0., 1., 0., 0., 1., 0.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0., 0., 1., 1.,
         1., 1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0.,
         0., 1., 1., 0., 0., 1., 1., 1., 

...

torch.Size([64])
tensor([[1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1.,
         1., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1.,
         1., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1.,
         0., 1., 0., 0., 1., 0., 1., 0., 0., 0.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1.,
         1., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1.,
         1., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1.,
         0., 1., 0., 0., 1., 0., 1., 0., 0., 0.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1.,
         1., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1.,
         1., 1., 0., 0., 1., 0., 0., 0., 

torch.Size([64])
tensor([[1., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1.,
         0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0.,
         0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0.,
         1., 1., 1., 0., 1., 0., 0., 0., 1., 1.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[1., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1.,
         0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0.,
         0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0.,
         1., 1., 1., 0., 1., 0., 0., 0., 1., 1.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[1., 1., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0.,
         0., 0., 1., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 1.,
         1., 0., 0., 1., 0., 0., 0., 0., 

torch.Size([64])
tensor([[1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 1.,
         0., 1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1.,
         0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0., 0., 0., 1.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 1.,
         0., 1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1.,
         0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0., 0., 0., 1.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[1., 1., 0., 0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0.,
         1., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1.,
         1., 1., 1., 0., 0., 0., 0., 0., 

torch.Size([64])
tensor([[1., 1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0.,
         0., 0., 0., 1., 1., 1., 0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1.,
         0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
         1., 0., 1., 0., 1., 0., 0., 0., 0., 0.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[1., 1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0.,
         0., 0., 0., 1., 1., 1., 0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1.,
         0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
         1., 0., 1., 0., 1., 0., 0., 0., 0., 0.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[1., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0.,
         0., 1., 1., 1., 0., 0., 0., 0., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1.,
         0., 0., 0., 1., 0., 1., 1., 1., 

torch.Size([64])
tensor([[1., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1.,
         0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0.,
         1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 1., 0., 1., 1.,
         0., 1., 1., 0., 0., 0., 0., 0., 1., 1.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[1., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1.,
         0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0.,
         1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 1., 0., 1., 1.,
         0., 1., 1., 0., 0., 0., 0., 0., 1., 1.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[1., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1.,
         0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0.,
         1., 0., 0., 1., 0., 0., 1., 0., 

In [47]:
!ls ../../data/small_data/train/1111100010001101110100100001010010101001001010001110110110000011

Tom_Hanks_0001.jpg  Tom_Hanks_0004.jpg	Tom_Hanks_0007.jpg  Tom_Hanks_0010.jpg
Tom_Hanks_0002.jpg  Tom_Hanks_0005.jpg	Tom_Hanks_0008.jpg
Tom_Hanks_0003.jpg  Tom_Hanks_0006.jpg	Tom_Hanks_0009.jpg


In [ ]:
path = '../../data/small_data/train/1111100010001101110100100001010010101001001010001110110110000011/'
for i in os.listdir(path):
        Image.open(path + i)
Image.open('../../data/small_data/train/1111100010001101110100100001010010101001001010001110110110000011/T')

In [ ]:
model.forward()